# LangChain v1.0 create_agent 解説ノートブック

このノートブックは、LangChain v1.0で導入された`create_agent`とミドルウェアの仕組みを実際に動かしながら理解するためのものです。

**動作環境**
- Python 3.12
- langchain 1.1.3
- langchain-anthropic 1.2.0
- langgraph 1.0.4

**注意**: このノートブックを実行するとAnthropic APIが呼び出され、料金が発生します。

## セクション1: 環境セットアップ

まず、必要なライブラリをインポートし、環境変数を読み込みます。
`.env`ファイルに`ANTHROPIC_API_KEY`が設定されている必要があります。

In [ ]:
from dotenv import load_dotenv
load_dotenv()

# 基本的なインポート
from langchain.agents import create_agent
from langchain.tools import tool

print("環境セットアップ完了!")

## セクション2: create_agentの基本

`create_agent`は、LangChain v1.0でAIエージェントを作成するための中心的なAPIです。
最もシンプルな形でエージェントを作成してみましょう。

In [ ]:
# 最もシンプルなエージェント (ツールなし)
simple_agent = create_agent(
    model="anthropic:claude-sonnet-4-5-20250929",
    tools=[],  # ツールなし
    system_prompt="あなたは親切なアシスタントです。"
)

# エージェントを実行
result = simple_agent.invoke({
    "messages": [{"role": "user", "content": "こんにちは！簡単に自己紹介してください。"}]
})

# 結果を確認
print("=== 結果の構造 ===")
print(f"キー: {result.keys()}")
print()
print("=== 最後のメッセージ ===")
last_message = result["messages"][-1]
print(f"タイプ: {type(last_message).__name__}")
print(f"内容: {last_message.content}")

## セクション3: toolsパラメータの指定方法

`@tool`デコレータを使用してツールを定義し、エージェントに渡します。
型ヒントとdocstringは必須で、これがモデルに渡されるツールの説明になります。

In [ ]:
@tool
def search_web(query: str) -> str:
    """Web検索を実行する"""
    return f"「{query}」の検索結果: Pythonは汎用プログラミング言語です。"

@tool
def get_weather(city: str) -> str:
    """指定した都市の天気を取得する"""
    return f"{city}の天気: 晴れ、気温25度"

# ツール付きエージェントを作成
agent_with_tools = create_agent(
    model="anthropic:claude-sonnet-4-5-20250929",
    tools=[search_web, get_weather],
    system_prompt="あなたは検索と天気情報を提供するアシスタントです。"
)

# ツールが呼ばれる質問をする
result = agent_with_tools.invoke({
    "messages": [{"role": "user", "content": "東京の天気を教えてください"}]
})

# 結果を確認
print("=== メッセージ履歴 ===")
for i, msg in enumerate(result["messages"]):
    msg_type = type(msg).__name__
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"{i}: [{msg_type}] ツール呼び出し: {[tc['name'] for tc in msg.tool_calls]}")
    elif hasattr(msg, 'name'):
        print(f"{i}: [{msg_type}] {msg.name}: {msg.content[:50]}...")
    else:
        content = msg.content if isinstance(msg.content, str) else str(msg.content)
        print(f"{i}: [{msg_type}] {content[:80]}...")

## セクション4: modelパラメータの指定方法

モデルは文字列またはモデルインスタンスで指定できます。

In [ ]:
from langchain_anthropic import ChatAnthropic

# 方法1: 文字列で指定
agent1 = create_agent(
    model="anthropic:claude-sonnet-4-5-20250929",
    tools=[get_weather],
)

# 方法1の実行テスト
result1 = agent1.invoke({
    "messages": [{"role": "user", "content": "京都の天気は?"}]
})
print("=== 方法1: 文字列で指定 ===")
print(f"応答: {result1['messages'][-1].content}")

# 方法2: ChatAnthropicインスタンスで指定 (詳細設定が可能)
model = ChatAnthropic(
    model="claude-sonnet-4-5-20250929",
    temperature=0,
    max_tokens=1024,
    timeout=30,
)

agent2 = create_agent(
    model=model,
    tools=[get_weather],
)

# 方法2の実行テスト
result2 = agent2.invoke({
    "messages": [{"role": "user", "content": "大阪の天気は?"}]
})
print("\n=== 方法2: ChatAnthropicインスタンスで指定 ===")
print(f"応答: {result2['messages'][-1].content}")

## セクション5: system_promptパラメータ

システムプロンプトは文字列または`SystemMessage`オブジェクトで指定できます。

In [ ]:
from langchain_core.messages import SystemMessage

# 方法1: 文字列で指定
agent_str_prompt = create_agent(
    model="anthropic:claude-sonnet-4-5-20250929",
    tools=[],
    system_prompt="あなたは関西弁で話すアシスタントです。",
)

result = agent_str_prompt.invoke({
    "messages": [{"role": "user", "content": "今日の調子はどう?"}]
})
print("=== 文字列でのシステムプロンプト ===")
print(result["messages"][-1].content)

# 方法2: SystemMessageで指定 (プロンプトキャッシュなどの高度な機能が使える)
# ※ cache_controlはAnthropicの機能で、大量のコンテキストをキャッシュできる
agent_sys_msg = create_agent(
    model="anthropic:claude-sonnet-4-5-20250929",
    tools=[],
    system_prompt=SystemMessage(
        content=[
            {"type": "text", "text": "あなたは敬語で話すアシスタントです。"},
            # 大量のコンテキスト情報がある場合、cache_controlを使うと効率的
            # {"type": "text", "text": "<大量のコンテキスト>", "cache_control": {"type": "ephemeral"}}
        ]
    ),
)

result = agent_sys_msg.invoke({
    "messages": [{"role": "user", "content": "今日の調子はどう?"}]
})
print("\n=== SystemMessageでのシステムプロンプト ===")
print(result["messages"][-1].content)

## セクション6: response_formatパラメータ

`response_format`を使用すると、AIエージェントの出力を構造化されたフォーマットで取得できます。

In [ ]:
from pydantic import BaseModel, Field
from langchain.agents.structured_output import ToolStrategy

class ContactInfo(BaseModel):
    """連絡先情報"""
    name: str = Field(description="名前")
    email: str = Field(description="メールアドレス")
    phone: str = Field(description="電話番号")

# 構造化出力を指定したエージェント
structured_agent = create_agent(
    model="anthropic:claude-sonnet-4-5-20250929",
    tools=[],
    response_format=ToolStrategy(ContactInfo),
)

result = structured_agent.invoke({
    "messages": [{"role": "user", "content": "田中太郎、tanaka@example.com、03-1234-5678"}]
})

print("=== 構造化された出力 ===")
print(f"structured_response: {result.get('structured_response')}")
print()
if result.get('structured_response'):
    contact = result['structured_response']
    print(f"名前: {contact.name}")
    print(f"メール: {contact.email}")
    print(f"電話: {contact.phone}")

## セクション7: 短期記憶 (checkpointer)

`checkpointer`を使用すると、会話履歴を保持して複数ターンの会話が可能になります。
`thread_id`ごとに会話が分離されます。

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

# checkpointer付きエージェント
memory_agent = create_agent(
    model="anthropic:claude-sonnet-4-5-20250929",
    tools=[],
    system_prompt="あなたは親切なアシスタントです。",
    checkpointer=InMemorySaver(),
)

# 会話1: 最初のメッセージ
config = {"configurable": {"thread_id": "conversation-001"}}

result1 = memory_agent.invoke(
    {"messages": [{"role": "user", "content": "私の名前は田中です。覚えておいてください。"}]},
    config=config,
)
print("=== ターン1 ===")
print(result1["messages"][-1].content)

# 会話2: 同じthread_idで続ける (前の会話を覚えている)
result2 = memory_agent.invoke(
    {"messages": [{"role": "user", "content": "私の名前を覚えていますか?"}]},
    config=config,
)
print("\n=== ターン2 (同じthread_id) ===")
print(result2["messages"][-1].content)

# 会話3: 異なるthread_idで実行 (新しい会話)
config_new = {"configurable": {"thread_id": "conversation-002"}}
result3 = memory_agent.invoke(
    {"messages": [{"role": "user", "content": "私の名前を覚えていますか?"}]},
    config=config_new,
)
print("\n=== ターン3 (別のthread_id) ===")
print(result3["messages"][-1].content)

## セクション8: ToolRuntimeの活用

`ToolRuntime`を使うと、ツール内から状態やコンテキストにアクセスできます。
`runtime`パラメータはモデルには見えず、実行時に自動で注入されます。

In [ ]:
from langchain.tools import ToolRuntime
from typing import TypedDict

# コンテキストスキーマを定義
class UserContext(TypedDict):
    user_name: str
    user_id: str

@tool
def get_user_info(runtime: ToolRuntime) -> str:
    """現在のユーザー情報を取得する"""
    user_name = runtime.context.get("user_name", "不明")
    user_id = runtime.context.get("user_id", "不明")
    return f"ユーザー名: {user_name}, ID: {user_id}"

@tool
def count_messages(runtime: ToolRuntime) -> str:
    """会話のメッセージ数を取得する"""
    messages = runtime.state.get("messages", [])
    return f"現在のメッセージ数: {len(messages)}件"

# context_schema付きエージェント
context_agent = create_agent(
    model="anthropic:claude-sonnet-4-5-20250929",
    tools=[get_user_info, count_messages],
    context_schema=UserContext,
    system_prompt="ツールを使ってユーザー情報を取得できます。",
)

# コンテキストを渡して実行
result = context_agent.invoke(
    {"messages": [{"role": "user", "content": "私のユーザー情報を教えてください"}]},
    context={"user_name": "田中太郎", "user_id": "user_123"},
)

print("=== ToolRuntimeでコンテキストにアクセス ===")
print(result["messages"][-1].content)

## セクション9: 長期記憶 (store)

`store`を使用すると、セッションを超えて永続化される情報を管理できます。
短期記憶(checkpointer)との違いは、スコープがセッションを超えて永続することです。

In [ ]:
from langgraph.store.memory import InMemoryStore

# ストアを作成
store = InMemoryStore()

# 事前にデータを保存
store.put(("users",), "user_123", {"name": "田中太郎", "preference": "Python"})

@tool
def get_stored_user_info(user_id: str, runtime: ToolRuntime) -> str:
    """ストアからユーザー情報を取得する"""
    info = runtime.store.get(("users",), user_id)
    if info:
        return f"ユーザー情報: {info.value}"
    return f"ユーザー {user_id} は見つかりませんでした"

@tool
def save_user_preference(user_id: str, preference: str, runtime: ToolRuntime) -> str:
    """ユーザーの好みをストアに保存する"""
    existing = runtime.store.get(("users",), user_id)
    data = existing.value if existing else {}
    data["preference"] = preference
    runtime.store.put(("users",), user_id, data)
    return f"ユーザー {user_id} の好みを '{preference}' に更新しました"

# store付きエージェント
store_agent = create_agent(
    model="anthropic:claude-sonnet-4-5-20250929",
    tools=[get_stored_user_info, save_user_preference],
    store=store,
    system_prompt="ユーザー情報の取得と保存ができます。",
)

# ユーザー情報を取得
result = store_agent.invoke({
    "messages": [{"role": "user", "content": "user_123の情報を教えてください"}]
})
print("=== ストアからの取得 ===")
print(result["messages"][-1].content)

## セクション10: state_schemaとcontext_schema

- `state_schema`: 実行中に変化する可能性のある情報 (AgentStateを継承)
- `context_schema`: 実行開始時に設定され、変化しない情報

In [ ]:
from langchain.agents import AgentState

# カスタムステートを定義 (AgentStateを継承)
class CustomState(AgentState):
    user_preferences: dict
    session_count: int

# カスタムコンテキストを定義
class AppContext(TypedDict):
    user_id: str
    environment: str  # "development" or "production"

@tool
def show_state_info(runtime: ToolRuntime) -> str:
    """現在の状態とコンテキスト情報を表示する"""
    prefs = runtime.state.get("user_preferences", {})
    count = runtime.state.get("session_count", 0)
    user_id = runtime.context.get("user_id", "不明")
    env = runtime.context.get("environment", "不明")
    return f"状態 - 設定: {prefs}, セッション数: {count} / コンテキスト - ユーザーID: {user_id}, 環境: {env}"

# state_schemaとcontext_schema両方を指定
custom_agent = create_agent(
    model="anthropic:claude-sonnet-4-5-20250929",
    tools=[show_state_info],
    state_schema=CustomState,
    context_schema=AppContext,
    system_prompt="状態とコンテキストの情報を表示できます。",
)

# 初期状態とコンテキストを渡して実行
result = custom_agent.invoke(
    {
        "messages": [{"role": "user", "content": "現在の状態情報を教えてください"}],
        "user_preferences": {"theme": "dark", "language": "ja"},
        "session_count": 5,
    },
    context={"user_id": "user_456", "environment": "production"},
)

print("=== state_schemaとcontext_schemaの確認 ===")
print(result["messages"][-1].content)

## セクション11: ミドルウェア - デコレータベース

ミドルウェアは、AIエージェントの実行フローに介入して動作を制御する仕組みです。
デコレータを使用して簡単に定義できます。

In [ ]:
from langchain.agents.middleware import before_model, after_model, dynamic_prompt, ModelRequest

@before_model
def log_before_model(state, runtime):
    """モデル呼び出し前にログを出力"""
    print(f"[BEFORE] モデル呼び出し開始: {len(state['messages'])}件のメッセージ")
    return None  # 状態を変更しない場合はNoneを返す

@after_model
def log_after_model(state, runtime):
    """モデル呼び出し後にログを出力"""
    print(f"[AFTER] モデル呼び出し完了")
    return None

# デコレータベースのミドルウェアを使用
logged_agent = create_agent(
    model="anthropic:claude-sonnet-4-5-20250929",
    tools=[],
    middleware=[log_before_model, log_after_model],
    system_prompt="あなたはアシスタントです。",
)

print("=== ミドルウェアのログを確認 ===")
result = logged_agent.invoke({
    "messages": [{"role": "user", "content": "こんにちは"}]
})
print(f"応答: {result['messages'][-1].content}")

In [ ]:
# @dynamic_promptを使用した動的プロンプト生成
@dynamic_prompt
def role_based_prompt(request: ModelRequest) -> str:
    """ユーザーの役割に応じてプロンプトを変更"""
    user_role = request.runtime.context.get("user_role", "user")
    if user_role == "admin":
        return "あなたは管理者向けアシスタントです。詳細な技術情報を提供してください。"
    elif user_role == "expert":
        return "あなたは専門家向けアシスタントです。専門用語を使って説明してください。"
    return "あなたは一般ユーザー向けアシスタントです。分かりやすく説明してください。"

class RoleContext(TypedDict):
    user_role: str

dynamic_agent = create_agent(
    model="anthropic:claude-sonnet-4-5-20250929",
    tools=[],
    context_schema=RoleContext,
    middleware=[role_based_prompt],
)

# 一般ユーザーとして質問
result_user = dynamic_agent.invoke(
    {"messages": [{"role": "user", "content": "APIとは何ですか?"}]},
    context={"user_role": "user"},
)
print("=== 一般ユーザー向け応答 ===")
print(result_user["messages"][-1].content[:200] + "...")

# 専門家として質問
result_expert = dynamic_agent.invoke(
    {"messages": [{"role": "user", "content": "APIとは何ですか?"}]},
    context={"user_role": "expert"},
)
print("\n=== 専門家向け応答 ===")
print(result_expert["messages"][-1].content[:200] + "...")

## セクション12: ミドルウェア - クラスベース

複数のフックを組み合わせたり、状態を保持する必要がある場合は、
`AgentMiddleware`クラスを継承してミドルウェアを作成します。

In [ ]:
from langchain.agents.middleware import AgentMiddleware, ModelRequest, ModelResponse
from typing import Callable, Any

class LoggingMiddleware(AgentMiddleware):
    """ロギング機能を持つカスタムミドルウェア"""
    
    def __init__(self, log_level: str = "INFO"):
        super().__init__()
        self.log_level = log_level
        self.call_count = 0
    
    def before_agent(self, state, runtime) -> dict[str, Any] | None:
        print(f"[{self.log_level}] AIエージェント開始")
        return None
    
    def wrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse]
    ) -> ModelResponse:
        self.call_count += 1
        print(f"[{self.log_level}] モデル呼び出し #{self.call_count}: {len(request.messages)}件のメッセージ")
        response = handler(request)
        print(f"[{self.log_level}] モデル応答受信")
        return response
    
    def after_agent(self, state, runtime) -> dict[str, Any] | None:
        print(f"[{self.log_level}] AIエージェント完了 (合計 {self.call_count} 回のモデル呼び出し)")
        return None

# クラスベースのミドルウェアを使用
class_middleware_agent = create_agent(
    model="anthropic:claude-sonnet-4-5-20250929",
    tools=[get_weather],
    middleware=[LoggingMiddleware(log_level="DEBUG")],
    system_prompt="天気情報を提供するアシスタントです。",
)

print("=== クラスベースミドルウェアの動作 ===")
result = class_middleware_agent.invoke({
    "messages": [{"role": "user", "content": "東京の天気を教えて"}]
})
print(f"\n応答: {result['messages'][-1].content}")

## セクション13: ModelRequestとModelResponse

`wrap_model_call`フックでは、`ModelRequest`と`ModelResponse`を操作できます。
`request.override()`を使用してリクエストを変更できます。

In [ ]:
from langchain.agents.middleware import wrap_model_call

@wrap_model_call
def add_context_to_request(request: ModelRequest, handler) -> ModelResponse:
    """リクエストにコンテキスト情報を追加"""
    # 現在時刻を追加情報としてメッセージに追加
    from datetime import datetime
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # 元のメッセージを取得
    original_messages = list(request.messages)
    
    # システムメッセージを追加 (最初のユーザーメッセージの前に)
    context_msg = {"role": "system", "content": f"現在時刻: {current_time}"}
    
    # メッセージリストを更新
    new_messages = [context_msg] + original_messages
    
    # リクエストを変更して実行
    modified_request = request.override(messages=new_messages)
    print(f"[wrap_model_call] コンテキスト追加: {current_time}")
    
    return handler(modified_request)

# wrap_model_callを使用するエージェント
wrap_agent = create_agent(
    model="anthropic:claude-sonnet-4-5-20250929",
    tools=[],
    middleware=[add_context_to_request],
    system_prompt="あなたは時間を意識したアシスタントです。",
)

result = wrap_agent.invoke({
    "messages": [{"role": "user", "content": "今何時ですか?"}]
})
print(f"応答: {result['messages'][-1].content}")

In [ ]:
# 動的モデル選択の例
from langchain_anthropic import ChatAnthropic

# 2つのモデルを用意
fast_model = ChatAnthropic(model="claude-haiku-4-5-20251001")
smart_model = ChatAnthropic(model="claude-sonnet-4-5-20250929")

@wrap_model_call
def select_model_by_complexity(request: ModelRequest, handler) -> ModelResponse:
    """メッセージの長さに応じてモデルを切り替え"""
    # 最後のユーザーメッセージを取得
    user_messages = [m for m in request.messages if hasattr(m, 'content') and getattr(m, 'type', None) == 'human']
    if user_messages:
        last_user_msg = user_messages[-1]
        content = last_user_msg.content if isinstance(last_user_msg.content, str) else str(last_user_msg.content)
        msg_length = len(content)
    else:
        msg_length = 0
    
    # メッセージが長い場合は高性能モデルを使用
    if msg_length > 100:
        print(f"[動的選択] 長いメッセージ ({msg_length}文字) -> claude-sonnet-4-5")
        modified = request.override(model=smart_model)
    else:
        print(f"[動的選択] 短いメッセージ ({msg_length}文字) -> claude-haiku-4-5")
        modified = request.override(model=fast_model)
    
    return handler(modified)

dynamic_model_agent = create_agent(
    model=fast_model,  # デフォルトはfast_model
    tools=[],
    middleware=[select_model_by_complexity],
    system_prompt="あなたはアシスタントです。",
)

# 短いメッセージ
print("=== 短いメッセージ ===")
result1 = dynamic_model_agent.invoke({
    "messages": [{"role": "user", "content": "こんにちは"}]
})
print(f"応答: {result1['messages'][-1].content}\n")

# 長いメッセージ
print("=== 長いメッセージ ===")
long_message = "私は最近プログラミングを始めました。特にPythonに興味があります。機械学習やデータサイエンスの分野で活用したいと考えています。初心者におすすめの学習リソースや、効率的な学習方法があれば教えてください。"
result2 = dynamic_model_agent.invoke({
    "messages": [{"role": "user", "content": long_message}]
})
print(f"応答: {result2['messages'][-1].content[:200]}...")

## セクション14: 標準ミドルウェアの紹介

LangChainは、一般的なユースケースに対応するミドルウェアを標準で提供しています。

| ミドルウェア | 説明 |
|-------------|------|
| SummarizationMiddleware | トークン制限に近づいたときに会話履歴を自動で要約する |
| HumanInTheLoopMiddleware | ツール呼び出し前に人間の承認を要求する |
| ModelCallLimitMiddleware | モデル呼び出し回数を制限し、過剰なコストを防ぐ |
| ToolCallLimitMiddleware | ツールの呼び出し回数を制限する |
| ModelFallbackMiddleware | メインモデルが失敗した場合に代替モデルに切り替える |
| PIIMiddleware | 個人情報（PII）を検出してマスキングまたはブロックする |
| TodoListMiddleware | AIエージェントにタスク計画と進捗管理の機能を提供する |
| LLMToolSelectorMiddleware | メインモデル呼び出し前に、LLMで関連するツールを選択する |
| ToolRetryMiddleware | ツール呼び出しの失敗時に指数バックオフで自動リトライする |
| ModelRetryMiddleware | モデル呼び出しの失敗時に指数バックオフで自動リトライする |
| LLMToolEmulatorMiddleware | テスト目的でツール実行をLLMでエミュレートする |
| ContextEditingMiddleware | 古いツール結果を削除してコンテキストを管理する |
| ShellToolMiddleware | AIエージェントにシェルコマンド実行機能を提供する |
| FilesystemFileSearchMiddleware | ファイルシステムに対するGlob/Grep検索機能を提供する |

In [ ]:
from langchain.agents.middleware import (
    SummarizationMiddleware,
    ModelCallLimitMiddleware,
    ToolRetryMiddleware,
)

# =====================================
# 1. SummarizationMiddleware
# =====================================
print("=" * 50)
print("1. SummarizationMiddleware")
print("=" * 50)

summarization_agent = create_agent(
    model="anthropic:claude-sonnet-4-5-20250929",
    tools=[],
    middleware=[
        SummarizationMiddleware(
            model="anthropic:claude-haiku-4-5-20251001",
            trigger=("tokens", 4000),
            keep=("messages", 10),
        )
    ],
    checkpointer=InMemorySaver(),
)

# 実行: 複数ターンの会話
config = {"configurable": {"thread_id": "summarization-test"}}
result = summarization_agent.invoke(
    {"messages": [{"role": "user", "content": "こんにちは、今日は良い天気ですね"}]},
    config=config,
)
print(f"応答: {result['messages'][-1].content}")

# =====================================
# 2. ModelCallLimitMiddleware
# =====================================
print("\n" + "=" * 50)
print("2. ModelCallLimitMiddleware")
print("=" * 50)

limited_agent = create_agent(
    model="anthropic:claude-sonnet-4-5-20250929",
    tools=[search_web],
    middleware=[
        ModelCallLimitMiddleware(
            thread_limit=10,
            run_limit=3,
            exit_behavior="end",
        )
    ],
)

result = limited_agent.invoke({
    "messages": [{"role": "user", "content": "Pythonについて教えて"}]
})
print(f"応答: {result['messages'][-1].content[:200]}...")

# =====================================
# 3. ToolRetryMiddleware
# =====================================
print("\n" + "=" * 50)
print("3. ToolRetryMiddleware")
print("=" * 50)

import random
random.seed(42)  # 再現性のためシードを固定

@tool
def unreliable_api(query: str) -> str:
    """不安定なAPIを呼び出す (テスト用)"""
    if random.random() < 0.3:  # 30%の確率で失敗
        raise ConnectionError("API接続エラー")
    return f"APIの結果: {query}に関する情報です"

retry_agent = create_agent(
    model="anthropic:claude-sonnet-4-5-20250929",
    tools=[unreliable_api],
    middleware=[
        ToolRetryMiddleware(
            max_retries=3,
            backoff_factor=2.0,
            initial_delay=0.1,
        )
    ],
)

result = retry_agent.invoke({
    "messages": [{"role": "user", "content": "最新のニュースを教えて"}]
})
print(f"応答: {result['messages'][-1].content[:200]}...")